# 1. 데이터 로드 및 전처리

In [ ]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/kor-eng.zip'
filename = 'kor-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [ ]:
num_samples = 3800

## 전처리 함수

In [ ]:
def preprocess_sentence_eng(sent):
  sent = sent.lower()

  # 단어와 구두점 사이에 공백 추가.
  # ex) "I am a student." => "I am a student ."
  sent = re.sub(r"([?.!,])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent

In [ ]:
def preprocess_sentence_kor(sent):

  sent = re.sub(r"([?.!,])", r" \1", sent)

  sent = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣!.?]+", r" ", sent)

  sent = re.sub(r"\s+", " ", sent)
  return sent

In [ ]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
kr_sent = u"저녁 먹었어?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence_eng(en_sent))
print('전처리 전 한국어 문장 :', kr_sent)
print('전처리 후 한국어 문장 :', preprocess_sentence_kor(kr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 한국어 문장 : 저녁 먹었어?
전처리 후 한국어 문장 : 저녁 먹었어 ?


In [ ]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("kor.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence_eng(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence_kor(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()]
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target

In [ ]:
sents_en_in, sents_kr_in, sents_kr_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_kr_in[:5])
print('디코더의 레이블 :',sents_kr_out[:5])

인코더의 입력 : [['go', '.'], ['hi', '.'], ['run', '!'], ['run', '.'], ['who', '?']]
디코더의 입력 : [['<sos>', '가', '.'], ['<sos>', '안녕', '.'], ['<sos>', '뛰어', '!'], ['<sos>', '뛰어', '.'], ['<sos>', '누구', '?']]
디코더의 레이블 : [['가', '.', '<eos>'], ['안녕', '.', '<eos>'], ['뛰어', '!', '<eos>'], ['뛰어', '.', '<eos>'], ['누구', '?', '<eos>']]


## 토크나이저, 정수 인코딩, 패딩

In [ ]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_kr = Tokenizer(filters="", lower=False)
tokenizer_kr.fit_on_texts(sents_kr_in)
tokenizer_kr.fit_on_texts(sents_kr_out)

decoder_input = tokenizer_kr.texts_to_sequences(sents_kr_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_kr.texts_to_sequences(sents_kr_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [ ]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (3800, 17)
디코더의 입력의 크기(shape) : (3800, 16)
디코더의 레이블의 크기(shape) : (3800, 16)


In [ ]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_kr.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 한국어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 2473, 한국어 단어 집합의 크기 : 5582


In [ ]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_kr.word_index
index_to_tar = tokenizer_kr.index_word

In [ ]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [2174 1881 3446 ... 1470 2051  156]


In [ ]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [ ]:
encoder_input[3021]

array([ 80,   2, 114,   5,   2, 131, 351,   8, 555,   1,   0,   0,   0,
         0,   0,   0,   0], dtype=int32)

In [ ]:
decoder_input[3021]

array([   2,   10,  400, 1419,  905, 1332,  330,    1,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

In [ ]:
decoder_target[3021]

array([  10,  400, 1419,  905, 1332,  330,    1,    3,    0,    0,    0,
          0,    0,    0,    0,    0], dtype=int32)

## 학습 데이터와 테스트 데이터 분리

In [1]:
n_of_val = int(3800*0.1)
print('테스트 데이터의 개수 :',n_of_val)

테스트 데이터의 개수 : 380


In [ ]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [ ]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (3420, 17)
훈련 target 데이터의 크기 : (3420, 16)
훈련 target 레이블의 크기 : (3420, 16)
테스트 source 데이터의 크기 : (380, 17)
테스트 target 데이터의 크기 : (380, 16)
테스트 target 레이블의 크기 : (380, 16)


# 2. 모델 학습

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [ ]:
embedding_dim = 64
hidden_units = 64

In [ ]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [ ]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
27/27 [==============================] - 22s 454ms/step - loss: 8.1394 - acc: 0.5499 - val_loss: 6.7352 - val_acc: 0.6146
Epoch 2/50
27/27 [==============================] - 9s 333ms/step - loss: 5.0981 - acc: 0.6134 - val_loss: 3.6414 - val_acc: 0.6146
Epoch 3/50
27/27 [==============================] - 9s 333ms/step - loss: 3.0602 - acc: 0.6134 - val_loss: 2.8180 - val_acc: 0.6146
Epoch 4/50
27/27 [==============================] - 9s 334ms/step - loss: 2.7094 - acc: 0.6134 - val_loss: 2.7262 - val_acc: 0.6146
Epoch 5/50
27/27 [==============================] - 9s 335ms/step - loss: 2.6201 - acc: 0.6134 - val_loss: 2.6696 - val_acc: 0.6146
Epoch 6/50
27/27 [==============================] - 9s 338ms/step - loss: 2.5479 - acc: 0.6134 - val_loss: 2.6039 - val_acc: 0.6146
Epoch 7/50
27/27 [==============================] - 9s 337ms/step - loss: 2.4587 - acc: 0.6134 - val_loss: 2.5127 - val_acc: 0.6150
Epoch 8/50
27/27 [==============================] - 9s 335ms/step - loss: 2

# 3. 모델 학습 결과

In [ ]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

## 학습 데이터의 번역 결과

In [ ]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : i have to give this book to tom . 
정답문장 : 톰에게 이 책을 줘야 합니다 . 
번역문장 : 난 내 프랑스어를 것을 않아 . 
--------------------------------------------------
입력문장 : they have no proof . 
정답문장 : 그들한텐 증거가 없어 . 
번역문장 : 너 ! 
--------------------------------------------------
입력문장 : tom held his breath . 
정답문장 : 톰은 숨을 참았어 . 
번역문장 : 톰은 자기 자기 톰이 톰이 비해 같아 . 
--------------------------------------------------
입력문장 : i m trying to do my best . 
정답문장 : 난 최선을 다하려고 노력하는 중이야 . 
번역문장 : 난 내 프랑스어를 것을 같아 . 
--------------------------------------------------
입력문장 : stop shooting . 
정답문장 : 총 그만 쏴 . 
번역문장 : ! 
--------------------------------------------------


## 테스트 데이터의 번역 결과

In [ ]:
for seq_index in [3, 50, 100, 300, 379]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : i know what those situations are like . 
정답문장 : 이게 대략 어떤 상황인지는 알아 . 
번역문장 : 너 좀 더 ? 
--------------------------------------------------
입력문장 : keep singing . 
정답문장 : 계속 노래해 . 
번역문장 : 너 있어 . 
--------------------------------------------------
입력문장 : it s awfully warm today . 
정답문장 : 오늘은 끔찍하게 덥네 . 
번역문장 : ! 
--------------------------------------------------
입력문장 : tom used to be a primary school teacher . 
정답문장 : 톰은 한때 초등학교 선생님이었었어 . 
번역문장 : 톰은 자기 자기 톰이 톰이 비해 같아 . 
--------------------------------------------------
입력문장 : seriously ? 
정답문장 : 진심이야 ? 
번역문장 : ! 
--------------------------------------------------


# 4. 한계점과 추후 해결방안